## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True

tradelogger.setLevel(logging.DEBUG)

In [2]:
tradelogger.info("Let's do trading")

##  <span style="color:green">Trading Strategy </span>

In [30]:
# ====== Tradescript Wrapper =======
# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


from talib import MIN, MAX, CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLLONGLINE, CDLDARKCLOUDCOVER, CDLPIERCING, CDLBELTHOLD, CDLCLOSINGMARUBOZU

class algoTrade:
   
    def __init__(s, price):
        logging.debug(price.tail(10))
        s.OPEN = price['open']
        s.CLOSE = price['close']
        s.HIGH = price['high']
        s.LOW = price['low']
        s.BBT, s.BBM, s.BBB = BBANDS( s.CLOSE, 20,1.75,1.75,0)
        s.BBT2, s.BBM2, s.BBB2 = BBANDS( s.CLOSE, 20,3,3,0)
        s.iMACD, s.iMACDsignal, s.iMACDhist = MACDEXT(s.CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        s.SD = STDDEV(s.CLOSE)
        s.fastk, s.fastd = STOCHF(s.HIGH, s.LOW, s.CLOSE)
        s.iRSI = RSI(s.CLOSE, timeperiod=20)
        s.STATE = ""
        s.profit = 0
        s.stoploss = 0
        s.openpositions = 0
        s.target = 0
        #logging.info(price)
    
    # Long Strategies
    def long_bb(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.iRSI,1) <=30)&(s.iRSI>30)
        temp_df = temp_df & ( CROSSOVER(s.CLOSE, s.BBB) & (CDLSPINNINGTOP(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 0))
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_bb(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80) & CROSSOVER(s.fastd, s.fastk)
        #temp_df = temp_df | CROSSOVER(fastd, fastk)
        temp_df = temp_df | (REF(s.iRSI,1) >=70)&(s.iRSI<70)
        temp_df = temp_df & CROSSOVER(s.BBT, s.CLOSE)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    
    def long_pa(s):
        temp_df = 0
        temp_df =   (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLPIERCING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHAMMER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | ((CDLMORNINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) & (CDLMORNINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100))
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) ==100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        
        temp_df = REF(TREND_DOWN(s.CLOSE,4),1) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(s):
        temp_df = 0
        
        temp_df =   (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDARKCLOUDCOVER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | (CDLHANGINGMAN(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | ((CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) & (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100))
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        temp_df = REF(TREND_UP(s.CLOSE,4),1) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_breakout(s):
        temp_df = s.HIGH >= MAX(s.HIGH, timeperiod=5)
       # print(pd.DataFrame({"high":s.HIGH, "max":MAX(s.HIGH, timeperiod=15), "t1": s.HIGH>= MAX(s.HIGH, timeperiod=15)
       #                     ,"low":s.LOW, "min":MIN(s.LOW, timeperiod=15)}))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.LOW <= MIN(s.LOW, timeperiod=5)
        return pd.DataFrame( temp_df , columns=["sell"])
    
  
    def tradeDecision(s):
        buy = s.long_breakout() #s.long_bb() & s.long_pa()
        sell = s.short_breakout() #s.short_bb()# & self.short_pa()
        
        buy['low'] = s.LOW
        sell['high'] = s.HIGH
        buy['close'] = -1 * s.CLOSE
        sell['close'] = s.CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        
        #temp_df = pd.DataFrame(buy.close)
        
        temp_df = pd.concat([buy.close, sell.close]).sort_index()
        #temp_df.append(buy.close)
        #temp_df = temp_df.append(sell.close)
        logging.debug(temp_df)
        return (buy,sell)
    
    # do not set timestamp for live trading
    def tradeBot(s, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)
        
        #atr = ATR(HIGH, LOW, CLOSE)[-1]
        s.atr = s.CLOSE[-1]*0.025

        price = s.CLOSE[-1]
        trigger = s.CLOSE[-1]
        longSL = toTick(s.CLOSE[-1] - s.atr)
        shortSL = toTick(s.CLOSE[-1]+s.atr)
        longTarget = toTick(s.CLOSE[-1] + s.atr) 
        shortTarget = toTick(s.CLOSE[-1]- s.atr)

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        if papertrade == True and STATE != "":
            if STATE == "SELL":
                if s.LOW[-1]<=s.stoploss:
                    temp_price = s.stoploss
                    s.STATE = ""
                    s.openpositions = s.openpositions - 1
                    tradelogger.info("Squareoff Stoploss: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif HIGH[-1]>=target:
                    temp_price = s.target
                    s.STATE = ""
                    s.openpositions = s.openpositions - 1
                    tradelogger.info("Squareoff Target: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
            elif STATE == "BUY":
                if HIGH[-1]>=stoploss:
                    temp_price = -1 * s.stoploss
                    s.STATE = ""
                    s.openpositions = s.openpositions + 1
                    tradelogger.info("Squareoff Stoploss: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif LOW[-1]<=target:
                    temp_price = -1 * s.target
                    s.STATE = ""
                    s.openpositions = s.openpositions + 1
                    tradelogger.info("Squareoff Target: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                
            
        
        buy, sell = s.tradeDecision()
        
        tradelogger.debug("STATE:" + s.STATE)
        tradelogger.debug(buy)
        tradelogger.debug(sell)
        
        
        tradelogger.debug(timeNow)
        tradelogger.debug(delT1)
        tradelogger.debug(delT2)
        
        # BUY in BUY state, SELL in STATE state
        if check_trade(buy, delT1, delT2) and (STATE != "SELL"):            
            tradelogger.info("buy:"+str(buy.index[-1]))
            s.openpositions = s.openpositions + 1
            s.stoploss = longSL
            target = longTarget
            tradelogger.debug(s.openpositions)
            
            qty = 1
            if s.openpositions != 0:
                qty = 2
                s.openpositions = s.openpositions + 1
            #tradelogger.info(atr)
            ##### Order Placement #####
            buy_bo(symbol, price, trigger, longSL, longTarget, quantity=qty)
            #buy_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "SELL"
            return -1 * price * qty
            
        elif check_trade(sell, delT1, delT2) and (STATE!="BUY"):                
            tradelogger.info("sell:"+str(sell.index[-1]))
            openpositions = openpositions - 1
            stoploss = shortSL
            target = shortTarget
            tradelogger.debug(s.openpositions)
            #tradelogger.info(atr)
            ##### Order Placement #####
            qty = 1
            if s.openpositions != 0:
                qty = 2
                s.openpositions = s.openpositions - 1
            sell_bo(symbol, price, trigger, shortSL, shortTarget, quantity=qty)
            #sell_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "BUY"
            return price * qty
        
        return 0

#portfolio = pd.DataFrame(downloadlist)

if mode == "Algo" and True:
    x = 'BHARTIARTL'
    
    toDate = dt.datetime(2019,4,22,15,15)
    fromDate =   getFromDate(toDate)
    #logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    temp_data = getData(x, fromDate, toDate, "minute")
    
    a = algoTrade(temp_data)
    
    buy, sell = a.tradeDecision()
    
    #print(buy, sell)
    
    fig = createPlot(x)
    plot_set1(temp_data)
    
    annotateBuySell(buy, "Buy")
    annotateBuySell(sell, "Sell")
    plotData(x, True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [8]:
tradelogger.setLevel(logging.INFO)
if mode == "Algo":
    #portfolio = downloadlist
    portfolio = ['BHARTIARTL']
    toDate = dt.datetime(2019,4,24,15,15)
    fromDate = getFromDate(toDate)
    STATE = ""
    stoploss = 0
    target = 0
    openpositions = 0
    portfolio = pd.DataFrame(portfolio)
    papertrade = True
    backtest(portfolio, fromDate, toDate)

BHARTIARTL: -5.8, (-1.7918 %)


- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)


In [12]:
import sys
from multiprocessing import Process
import time

def f(name):
    print('hello', name)
    logger.debug("Hello")

if __name__ == '__main__':
    logger.debug("hello")
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()
    sys.stdout.flush()
    time.sleep(5)

In [26]:
import test
from multiprocessing import Pool

if __name__ ==  '__main__': #this is the only additional line
    num_processors = 3
    p=Pool(processes = num_processors)
    output = p.map(test.worker,[i for i in range(0,3)])
    print(output)

[0, 1, 4]


In [21]:
class A:
    varA = 15
    def disp2(self):
        print("A: ",self.varA)
    pass

class B(A):
    def disp(self):
        self.disp2()
        print("B :",self.varA)
        
b = B()
b.disp()

A:  15
B : 15
